In [39]:
# import
import pandas as pd
import numpy as np
import gensim as gs
import csv
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, LabeledSentence
import gensim.models.doc2vec
import os
import logging
from gensim.models import Doc2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
print('import over')

import over


In [53]:
# 文本预处理
print('start')
csvDataTrain = pd.read_csv('C:\\Users\\ThinkPad\\Desktop\\bishe\\KDDTrain.csv')  # 读取训练数据
arrCsvTrain = np.array(csvDataTrain)
csvDataTest = pd.read_csv('C:\\Users\\ThinkPad\\Desktop\\bishe\\KDDTest.csv')  # 读取测试数据
arrCsvTest = np.array(csvDataTest)
docTrain = arrCsvTrain[:,:-1]
tagTrain = arrCsvTrain[:,-1]
docTest = arrCsvTest[:,:-1]
tagTest = arrCsvTest[:,-1]
docstest = []
docs = []
for i in range(len(docTrain)):
    docs.append(gs.models.doc2vec.TaggedDocument(' '.join(docTrain[i]).split(' '),[str(tagTrain[i])]))
for i in range(len(docTest)):
    docstest.append(gs.models.doc2vec.TaggedDocument(' '.join(docTest[i]).split(' '), [str(tagTest[i])]))

print('读入文档结束')

start
读入文档结束


In [55]:
#   加载函数
'''      获得向量函数        '''
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors
'''      余弦相似度预测函数     '''
def Calcossim(token,X_test,y_test):
    y_pred_cos = []
    for i in range(len(X_test)):
        maxdis = -1
        cur = -1
        for j in range(5):
            dis = float(np.dot(X_test[i], token[j]) / (np.linalg.norm(X_test[i]) * np.linalg.norm(token[j])))
            if maxdis < dis:
                maxdis = dis
                cur = j
        y_pred_cos.append(str(cur))
    num_acc = 0
   
    with open('C:\\Users\\ThinkPad\\Desktop\\bishe\\1.txt', 'w', newline='') as file:
        for i in range(len(y_pred_cos)):
            file.write(str(y_pred_cos[i]))
    with open('C:\\Users\\ThinkPad\\Desktop\\bishe\\2.txt', 'w', newline='') as file:
        for i in range(len(y_test)):
            file.write(str(y_test[i]))
    for i in range(len(y_pred_cos)):
        if y_test[i] == y_pred_cos[i]:
            num_acc+=1
    print("余弦相似度正确率为：{}".format(num_acc/len(y_pred_cos)))

'''      逻辑回归预测函数      '''
def Callogissim(X_train,y_train,X_test,y_test):
    logreg = LogisticRegression(n_jobs=1, C=1e5)
    logreg.fit(X_train, y_train)
    y_pred_logis = logreg.predict(X_test)
    print('逻辑回归Testing accuracy :%s' % accuracy_score(y_test, y_pred_logis))
    print('逻辑回归Testing F1 score: {}'.format(f1_score(y_test, y_pred_logis, average='weighted')))  #labels=np.unique(y_pred)

print('加载函数结束')

加载函数结束


In [54]:
#  训练并保存 dbow模型
print('start dbow train')
size_dbow = 50
cores = multiprocessing.cpu_count()
docmodel_dbow = gensim.models.Doc2Vec(dm=0, min_count=1, workers=cores, vector_size = size_dbow)
docmodel_dbow.build_vocab(docs)
#docmodel.train(docs, total_examples=docmodel.corpus_count, epochs=docmodel.iter)
for epoch in range(5):
    docmodel_dbow.train(docs, total_examples=len(docs), epochs=5)
    docmodel_dbow.alpha -= 0.0025
    docmodel_dbow.min_alpha = docmodel_dbow.alpha
print("dbow over")
docmodel_dbow.save('C:\\Users\\ThinkPad\\Desktop\\bishe\\model\\docmodel__dbow__standard__cos_and_logis_ipynb.model')


2019-12-07 13:47:05,506 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-12-07 13:47:05,508 : INFO : collecting all words and their counts
2019-12-07 13:47:05,509 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-12-07 13:47:05,628 : INFO : PROGRESS: at example #10000, processed 380000 words (3233838/s), 148 word types, 1 tags
2019-12-07 13:47:05,733 : INFO : PROGRESS: at example #20000, processed 760000 words (3642847/s), 148 word types, 1 tags
2019-12-07 13:47:06,003 : INFO : PROGRESS: at example #30000, processed 1140000 words (1412048/s), 148 word types, 1 tags
2019-12-07 13:47:06,103 : INFO : PROGRESS: at example #40000, processed 1520000 words (3886275/s), 148 word types, 1 tags
2019-12-07 13:47:06,198 : INFO : PROGRESS: at example #50000, processed 1900000 words (4005257/s), 149 word types, 1 tags
2019-12-07 13:47:06,294 : INFO : PROGRESS: at example #60000, processed 2280000 words (4001002/s), 149 word ty

start dbow train
dbow over


In [56]:
print('load dbow model')
#加载dbow模型，获得向量，预测结果，显示结果  目前74
docmodel_dbow = gs.models.Doc2Vec.load('C:\\Users\\ThinkPad\\Desktop\\bishe\\model\\docmodel__dbow__standard__cos_and_logis.model')
y_train_dbow, X_train_dbow = vec_for_learning(docmodel_dbow, docs)
y_test_dbow, X_test_dbow = vec_for_learning(docmodel_dbow, docstest)
# 获得标签向量
token_dbow = []
for i in range(5):
    token_dbow.append(docmodel_dbow.docvecs[i])
# dbow 余弦相似度预测
print('dbow :',end='')
Calcossim(token_dbow,X_test_dbow,y_test_dbow)
# dbow 逻辑回归预测
print('dbow :',end='')
Callogissim(X_train_dbow,y_train_dbow,X_test_dbow,y_test_dbow)

2019-12-07 13:51:01,062 : INFO : loading Doc2Vec object from C:\Users\ThinkPad\Desktop\bishe\model\docmodel__dbow__standard__cos_and_logis_ipynb.model
2019-12-07 13:51:01,065 : INFO : loading vocabulary recursively from C:\Users\ThinkPad\Desktop\bishe\model\docmodel__dbow__standard__cos_and_logis_ipynb.model.vocabulary.* with mmap=None
2019-12-07 13:51:01,066 : INFO : loading trainables recursively from C:\Users\ThinkPad\Desktop\bishe\model\docmodel__dbow__standard__cos_and_logis_ipynb.model.trainables.* with mmap=None
2019-12-07 13:51:01,066 : INFO : loading wv recursively from C:\Users\ThinkPad\Desktop\bishe\model\docmodel__dbow__standard__cos_and_logis_ipynb.model.wv.* with mmap=None
2019-12-07 13:51:01,067 : INFO : loading docvecs recursively from C:\Users\ThinkPad\Desktop\bishe\model\docmodel__dbow__standard__cos_and_logis_ipynb.model.docvecs.* with mmap=None
2019-12-07 13:51:01,068 : INFO : loaded C:\Users\ThinkPad\Desktop\bishe\model\docmodel__dbow__standard__cos_and_logis_ipynb

load dbow model
dbow :余弦相似度正确率为：0.0
dbow :逻辑回归Testing accuracy :0.42261455884309984
逻辑回归Testing F1 score: 0.2701983983119802


In [33]:
#  训练并保存 dm模型
size_dm = 16
cores = multiprocessing.cpu_count()

docmodel_dm = gensim.models.Doc2Vec(alpha=0.025, dm_mean=None, dm=1, min_count=1, window=2, vector_size=16,
                       sample=1e-3, min_alpha=0.0001,epochs=50, negative=5, workers=4)
docmodel_dm.build_vocab(docs)
docmodel_dm.train(docs,total_examples=len(docs),epochs=docmodel_dm.epochs)
print("dm over")
docmodel_dm.save('C:\\Users\\ThinkPad\\Desktop\\bishe\\model\\docmodel__dm__standard__cos_and_logis_ipynb.model')

2019-12-07 12:49:50,648 : INFO : collecting all words and their counts
2019-12-07 12:49:50,650 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-12-07 12:49:50,736 : INFO : PROGRESS: at example #10000, processed 380000 words (4481269/s), 148 word types, 1 tags
2019-12-07 12:49:50,804 : INFO : PROGRESS: at example #20000, processed 760000 words (5610653/s), 148 word types, 1 tags
2019-12-07 12:49:50,880 : INFO : PROGRESS: at example #30000, processed 1140000 words (5095438/s), 148 word types, 1 tags
2019-12-07 12:49:50,994 : INFO : PROGRESS: at example #40000, processed 1520000 words (3359338/s), 148 word types, 1 tags
2019-12-07 12:49:51,094 : INFO : PROGRESS: at example #50000, processed 1900000 words (3813691/s), 149 word types, 1 tags
2019-12-07 12:49:51,179 : INFO : PROGRESS: at example #60000, processed 2280000 words (4527782/s), 149 word types, 1 tags
2019-12-07 12:49:51,247 : INFO : PROGRESS: at example #70000, processed 2660000 words (5629346/

dm over


In [57]:
#   加载dm模型
docmodel_dm = gs.models.Doc2Vec.load('C:\\Users\\ThinkPad\\Desktop\\bishe\\model\\docmodel__dm__standard__cos_and_logis.model')
#   获得标签向量
token_dm = []
for i in range(5):
    token_dm.append(docmodel_dm.docvecs[i])

y_train_dm, X_train_dm = vec_for_learning(docmodel_dm, docs)
y_test_dm, X_test_dm = vec_for_learning(docmodel_dm, docstest)
print('dm over')
# dm 余弦相似度预测
print('dm :',end='')
Calcossim(token_dm,X_test_dm,y_test_dm)
# dm 逻辑回归预测m
print('dm :',end='')
Callogissim(X_train_dm,y_train_dm,X_test_dm,y_test_dm)

2019-12-07 16:07:55,389 : INFO : loading Doc2Vec object from C:\Users\ThinkPad\Desktop\bishe\model\docmodel__dm__standard__cos_and_logis.model
D:\anaconda\envs\chen\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-12-07 16:07:55,416 : INFO : loading vocabulary recursively from C:\Users\ThinkPad\Desktop\bishe\model\docmodel__dm__standard__cos_and_logis.model.vocabulary.* with mmap=None
2019-12-07 16:07:55,417 : INFO : loading trainables recursively from C:\Users\ThinkPad\Desktop\bishe\model\docmodel__dm__standard__cos_and_logis.model.trainables.* with mmap=None
2019-12-07 16:07:55,418 : INFO : loading wv recursively from C:\Users\ThinkPad\Desktop\bishe\model\docmodel__dm__standard__cos_and_logis.model.w

dm over
dm :余弦相似度正确率为：0.0
dm :逻辑回归Testing accuracy :0.41569445060550947
逻辑回归Testing F1 score: 0.30186094996743834


In [9]:
#   两个模型结合
new_model = ConcatenatedDoc2Vec([docmodel_dbow, docmodel_dm])
# 获得标签向量
token_new = []
for i in range(5):
    token_new.append(new_model.docvecs[str(i)])
y_train_new, X_train_new = vec_for_learning(new_model, docs)
y_test_new, X_test_new = vec_for_learning(new_model, docstest)
print('newmodel over')
'''暂时不能预测  维数不一致'''
# # new 预测
# print('new :')
# Calcossim(token_new,X_test_new,y_test_new)
# new 逻辑回归预测
print('new :',end='')
Callogissim(X_train_new,y_train_new,X_test_new,y_test_new)


newmodel over
new :逻辑回归Testing accuracy :0.7196468970412101
逻辑回归Testing F1 score: 0.6749767598444902


D:\anaconda\envs\chen\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\anaconda\envs\chen\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [11]:
print("start")
#  段向量加和求平均 和 标签向量结合 提高2%左右
def Caltokendoc(tags,X_train,size):
    num_doc = [0]* 5
    token_doc = np.zeros((5,size))
    for i in range(len(tags)):
        index = int(tags[i])
        token_doc[index] += X_train[i]
        num_doc[index] += 1
    for i in range(5):
        token_doc[i] = token_doc[i]/num_doc[i]
    return token_doc

token_doc_dbow = Caltokendoc(tagTrain,X_train_dbow,size_dbow)
token_doc_dm = Caltokendoc(tagTrain,X_train_dm,size_dm)

# dbow 的向量和
token_token_doc_dbow = [0] * 5
for rate in range(0,101,10):
    for i in range(5):
        token_token_doc_dbow[i] = rate/100.0*token_doc_dbow[i]+(1-rate/100)*token_dbow[i]     #   查看相似度
    #   预测，求正确率
    Calcossim(token_token_doc_dbow,X_test_dbow,y_test_dbow)

  
token_token_doc_dm = [0] * 5
for rate in range(0,101,10):
    for i in range(5):
        token_token_doc_dm[i] = rate/100.0*token_doc_dm[i]+(1-rate/100)*token_dm[i]
    Calcossim(token_token_doc_dm,X_test_dm,y_test_dm)

start
余弦相似度正确率为：0.7010601960697334
余弦相似度正确率为：0.7006165993878366
余弦相似度正确率为：0.6999068446968016
余弦相似度正确率为：0.699019651333008
余弦相似度正确率为：0.6981324579692144
余弦相似度正确率为：0.6987091336556802
余弦相似度正确率为：0.6982211773055937
余弦相似度正确率为：0.696934746928093
余弦相似度正确率为：0.6956926762187818
余弦相似度正确率为：0.6903251563678303
余弦相似度正确率为：0.6839373641485161
余弦相似度正确率为：0.13445415428292595
余弦相似度正确率为：0.1323692498780109
余弦相似度正确率为：0.13050614381404427
余弦相似度正确率为：0.12744532670895622
余弦相似度正确率为：0.12540478197223084
余弦相似度正确率为：0.12243268420352216
余弦相似度正确率为：0.118129796389123
余弦相似度正确率为：0.11622233065696669
余弦相似度正确率为：0.12167856984429756
余弦相似度正确率为：0.17664019873131348
余弦相似度正确率为：0.46772834139200636


In [13]:
print('start')
#  词向量加和求平均 和 标签向量结合 提高   左右
def Caltokenword(tags,X_train,model,size):
    num_word = [0]* 5
    token_word = np.zeros((5,size))
    for i in range(len(tags)):
        index = int(tags[i])
        words = docTrain[i].split()
        for j in range(len(words)):
            token_word[index] += model.wv.get_vector(words[j])
            num_word[index] += 1
    for i in range(5):
        token_word[i] = token_word[i]/num_word[i]
    return token_word

token_word_dbow = Caltokenword(tagTrain,X_train_dbow,docmodel_dbow,size_dbow)
token_word_dm = Caltokenword(tagTrain,X_train_dm,docmodel_dm,size_dm)

#获得新的token ：token_token_word
token_token_word_dbow = [0] * 5
for rate in range(0,101,10):
    for i in range(5):
        token_token_word_dbow[i] = rate/100.0*token_word_dbow[i]+(1-rate/100)*token_dbow[i]
    Calcossim(token_token_word_dbow,X_test_dbow,y_test_dbow)
token_token_word_dm = [0] * 5
for rate in range(0,101,10):
    for i in range(5):
        token_token_word_dm[i] = rate/100.0*token_word_dm[i]+(1-rate/100)*token_dm[i]
    Calcossim(token_token_word_dm,X_test_dm,y_test_dm)

start
余弦相似度正确率为：0.7010601960697334
余弦相似度正确率为：0.7010601960697334
余弦相似度正确率为：0.7011045557379231
余弦相似度正确率为：0.7011045557379231
余弦相似度正确率为：0.7011045557379231
余弦相似度正确率为：0.7010601960697334
余弦相似度正确率为：0.7010158364015437
余弦相似度正确率为：0.7010158364015437
余弦相似度正确率为：0.7010158364015437
余弦相似度正确率为：0.7008827573969747
余弦相似度正确率为：0.44377412056957816
余弦相似度正确率为：0.13445415428292595
余弦相似度正确率为：0.13569622499223705
余弦相似度正确率为：0.13795856806991083
余弦相似度正确率为：0.13986603380206716
余弦相似度正确率为：0.1431930089162933
余弦相似度正确率为：0.148116932085348
余弦相似度正确率为：0.15690014638690503
余弦相似度正确率为：0.1997072261899481
余弦相似度正确率为：0.33757707492347955
余弦相似度正确率为：0.3414807257241716
余弦相似度正确率为：0.12465066761300625
